<a href="https://colab.research.google.com/github/mannmoshe/text-recognition/blob/main/true_text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import random

In [2]:
req = requests.get("https://raw.githubusercontent.com/mannmoshe/text-recognition/main/torah_heb.txt")
req.encoding = 'ISO-8859-8'
torah_text = req.text

In [3]:
torah_words = torah_text.split()

In [4]:
phrases = {}
for i in range(0, int(len(torah_words)), 3):
  phrase = ''.join(torah_words[i: i+3])
  phrases[phrase] = 1 # true text
  phrase_letters_list = [l for l in phrase]
  random.Random(4).shuffle(phrase_letters_list) 
  # Random(4) for same result every time, see https://stackoverflow.com/questions/19306976/python-shuffling-with-a-parameter-to-get-the-same-result
  phrases[''.join(phrase_letters_list)] = 0 # random text

In [5]:
phrases_dataset = pd.DataFrame.from_dict(phrases, orient='index').reset_index()

In [6]:
phrases_dataset.columns = ['text', 'label']

In [27]:
phrases_dataset.shape

(48974, 2)

In [7]:
phrases_dataset.head(10)

,text,label
0,בראשיתבראאלהים,1
1,איתאםלבהארבריש,0
2,אתהשמיםואת,1
3,אהאוםתיתמש,0
4,הארץוהארץהיתה,1
5,תיץההההרראאוץ,0
6,תהוובהווחשך,1
7,והושתחךוהבו,0
8,עלפניתהום,1
9,לפהוםתעינ,0


In [8]:
X = phrases_dataset['text']
y = phrases_dataset['label']

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4, stratify=y)

In [26]:
len(x_train)

32812

In [28]:
len(x_test)

16162

In [29]:
len(x_train) + len(x_test)

48974

In [10]:
train_samples = x_train.tolist()
test_samples = x_test.tolist()

In [11]:
import numpy as np
from keras.preprocessing.text import Tokenizer

In [17]:
# We create a tokenizer, configured to only take
# into account the top-10000 most common words
#tokenizer = Tokenizer(num_words=10000, char_level=True) # char_level = True --> token by letters
tokenizer = Tokenizer(char_level=True) # char_level = True --> token by letters
# This builds the word index
tokenizer.fit_on_texts(train_samples)

# This turns strings into lists of integer indices.
train_sequences = tokenizer.texts_to_sequences(train_samples)
test_sequences = tokenizer.texts_to_sequences(test_samples)

# You could also directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
x_train = tokenizer.texts_to_matrix(train_samples, mode='binary')
x_test = tokenizer.texts_to_matrix(test_samples, mode='binary')

# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 27 unique tokens.


In [18]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [25]:
x_train.shape

(32812, 28)

In [19]:
len(x_train[0])

28

In [22]:
train_samples[0]

'ישחםתמאשותיחמ'

In [21]:
x_train[0]

array([0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
sum(x_train[0])

8.0

In [32]:
word_index

{'י': 1,
 'ו': 2,
 'ה': 3,
 'א': 4,
 'ל': 5,
 'ת': 6,
 'ר': 7,
 'ב': 8,
 'ש': 9,
 'מ': 10,
 'ע': 11,
 'ם': 12,
 'נ': 13,
 'כ': 14,
 'ח': 15,
 'ד': 16,
 'ק': 17,
 'ן': 18,
 'פ': 19,
 'ך': 20,
 'צ': 21,
 'ז': 22,
 'ג': 23,
 'ס': 24,
 'ט': 25,
 'ץ': 26,
 'ף': 27}

In [33]:
lens = [len(s) for s in train_sequences]
lens[:20]

[13, 9, 11, 9, 10, 11, 15, 13, 13, 14, 12, 10, 14, 14, 10, 13, 11, 8, 11, 10]

In [34]:
sum(lens)/len(lens)

11.49652566134341

In [35]:
train_sequences[:10]

[[1, 9, 15, 12, 6, 10, 4, 9, 2, 6, 1, 15, 10],
 [1, 3, 1, 3, 8, 2, 10, 2, 12],
 [7, 3, 9, 1, 22, 17, 12, 10, 14, 7, 9],
 [12, 14, 5, 2, 3, 6, 4, 27, 24],
 [17, 8, 7, 2, 4, 6, 4, 8, 1, 2],
 [8, 10, 24, 19, 7, 9, 10, 6, 10, 8, 18],
 [6, 9, 10, 11, 2, 18, 4, 6, 3, 10, 9, 19, 25, 1, 12],
 [4, 7, 3, 2, 15, 2, 10, 7, 17, 2, 4, 7, 26],
 [9, 5, 1, 7, 20, 14, 4, 1, 19, 2, 8, 5, 10],
 [3, 8, 10, 10, 2, 2, 8, 9, 4, 15, 5, 7, 12, 6]]

In [36]:
max_len = 12
x_train_embedding = []

for s in train_sequences:
  if len(s) >= 12:
    x_train_embedding.append(s[:12])
  else:
    x_train_embedding.append(s + [0]*(max_len-len(s)))

x_train_embedding = np.array(x_train_embedding)

In [37]:
x_test_embedding = []

for s in test_sequences:
  if len(s) >= 12:
    x_test_embedding.append(s[:12])
  else:
    x_test_embedding.append(s + [0]*(max_len-len(s)))

x_test_embedding = np.array(x_test_embedding)

In [38]:
y_train = y_train.to_numpy()

In [39]:
y_train

array([0, 1, 0, ..., 0, 0, 0])

In [40]:
x_train_embedding

array([[ 1,  9, 15, ...,  6,  1, 15],
       [ 1,  3,  1, ...,  0,  0,  0],
       [ 7,  3,  9, ...,  7,  9,  0],
       ...,
       [ 1,  8,  5, ..., 14,  0,  0],
       [10,  1,  9, ..., 13,  8,  0],
       [ 7,  3,  4, ..., 13,  0,  0]])

In [41]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import SimpleRNN, Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [42]:
model = Sequential()
model.add(Embedding(10000, 8, input_length=max_len))
model.add(LSTM(64))
model.compile(optimizer='rmsprop',
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.add(Dense(1, activation="sigmoid"))
model.summary()

history = model.fit(x_train_embedding, 
                    y_train,
                    epochs=50,
                    batch_size=64,
                    validation_split=0.33)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 8)             80000     
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 98,753
Trainable params: 98,753
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
344/344 [==============================] - 7s 14ms/step - loss: 0.6568 - accuracy: 0.5888 - val_loss: 0.6253 - val_accuracy: 0.6354
Epoch 2/50
344/344 [==============================] - 4s 12ms/step - loss: 0.6215 - accuracy: 0.6392 - val_loss: 0.6195 - val_accuracy: 0.6421
Epoch 3/50
344/344 [==========

In [43]:
x_train[0]

array([0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [44]:
predicted_labels = model.predict(x_test_embedding)

506/506 [==============================] - 2s 3ms/step


In [45]:
print(type(predicted_labels),type(y_test))

<class 'numpy.ndarray'> <class 'pandas.core.series.Series'>


In [46]:
predicted_labels

array([[0.999386  ],
       [0.8073914 ],
       [0.9961702 ],
       ...,
       [0.9842679 ],
       [0.02593355],
       [0.9952114 ]], dtype=float32)

In [47]:
my_list = map(lambda x: x[0], predicted_labels)
predictions = pd.Series(my_list)

In [48]:
len(predictions)

16162

In [49]:
len(y_test)

16162

In [50]:
predictions

0        0.999386
1        0.807391
2        0.996170
3        0.997528
4        0.957968
           ...   
16157    0.479061
16158    0.048900
16159    0.984268
16160    0.025934
16161    0.995211
Length: 16162, dtype: float32

In [51]:
y_test_ri = y_test.reset_index() 

In [52]:
y_test_ri

,index,label
0,15542,1
1,14953,0
2,20832,1
3,20498,1
4,1782,1
...,...,...
16157,26075,0
16158,27721,0
16159,36666,1
16160,41767,0


In [81]:
predictions_temp = pd.concat([predictions, y_test_ri], axis=1)

In [82]:
predictions_temp.head()

,0,index,label
0,0.999386,15542,1
1,0.807391,14953,0
2,0.996170,20832,1
3,0.997528,20498,1
4,0.957968,1782,1


In [54]:
x_train_orig, x_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.33, random_state=4, stratify=y)

In [55]:
x_test_orig

15542      מאדםועדבהמה
14953       דךמיהךטטנו
20832      ואכלואתםאשר
20498    ומשחתאתםומלאת
1782        עשבנתתילכם
             ...      
26075    הנמוגיותלבאחה
27721      ישחתעאולללו
36666      הארץכיידעתי
41767     רתושוואיתאצמ
12782       מזקןלאיוכל
Name: text, Length: 16162, dtype: object

In [96]:
x_test_orig.reset_index().head()

,index,text
0,15542,מאדםועדבהמה
1,14953,דךמיהךטטנו
2,20832,ואכלואתםאשר
3,20498,ומשחתאתםומלאת
4,1782,עשבנתתילכם


In [94]:
output_df = predictions_temp.merge(x_test_orig.reset_index(), how='inner', left_on='index', right_on='index')

In [95]:
output_df.head()

,0,index,label,text
0,0.999386,15542,1,מאדםועדבהמה
1,0.807391,14953,0,דךמיהךטטנו
2,0.996170,20832,1,ואכלואתםאשר
3,0.997528,20498,1,ומשחתאתםומלאת
4,0.957968,1782,1,עשבנתתילכם


In [102]:
output_df['prediction'] = output_df[0].round(0).astype('int')

In [103]:
output_df.head()

,0,index,label,text,prediction
0,0.999386,15542,1,מאדםועדבהמה,1
1,0.807391,14953,0,דךמיהךטטנו,1
2,0.996170,20832,1,ואכלואתםאשר,1
3,0.997528,20498,1,ומשחתאתםומלאת,1
4,0.957968,1782,1,עשבנתתילכם,1


In [105]:
sum(output_df['label'] == output_df['prediction']) / len(output_df['label']) # accuracy

0.8299096646454647

In [106]:
test1 = output_df.copy()

In [107]:
confusion_matrix = pd.crosstab(test1['label'], test1['prediction'], rownames=['Actual'], colnames=['Predicted'])

In [108]:
confusion_matrix

Predicted,0,1
Actual,,
0,5841,2240
1,509,7572


In [112]:
test1.rename(columns={0: "prediction_probability"}, inplace=True)

In [113]:
test1.columns

Index(['prediction_probability', 'index', 'label', 'text', 'prediction'], dtype='object')

In [114]:
test1_output = test1[['text', 'label', 'prediction', 'prediction_probability']]

In [115]:
test1_output.head()

,text,label,prediction,prediction_probability
0,מאדםועדבהמה,1,1,0.999386
1,דךמיהךטטנו,0,1,0.807391
2,ואכלואתםאשר,1,1,0.996170
3,ומשחתאתםומלאת,1,1,0.997528
4,עשבנתתילכם,1,1,0.957968


In [117]:
from google.colab import files

test1_output.to_csv('RNNtest1.csv', index=False ,encoding = 'utf-8-sig') 
files.download('RNNtest1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [125]:
req = requests.get("https://github.com/mannmoshe/text-recognition/raw/main/torah_skips_of_2.txt")
#req.encoding = 'ISO-8859-8'
torah_skips_of_2 = req.text

In [126]:
torah_skips_of_2[:10]

'באיבאליאהמ'

In [145]:
phrases_12 = {}

for i in range(0, len(torah_skips_of_2) - 12, 12):
  word = torah_skips_of_2[i:i + 12]
  phrases_12[word] = [i*2, i*2 + 12]

In [146]:
phrases_12_df = pd.DataFrame.from_dict(dict(phrases_12.items()), orient='index').reset_index()

In [147]:
phrases_12_df.columns = ['text', 'start_index', 'end_index']

In [148]:
phrases_12_df.head()

,text,start_index,end_index
0,באיבאליאהמםא,0,12
1,הרואץיההוהוש,24,36
2,עפיהםרחלימחת,48,60
3,לנהיוארליייו,72,84
4,והאריאליאהוכ,96,108


In [149]:
phrases_12_df.shape

(12671, 3)

In [154]:
X = phrases_12_df['text']

In [155]:
X_samples = X.tolist()

In [156]:
X_samples[:10]

['באיבאליאהמםא',
 'הרואץיההוהוש',
 'עפיהםרחלימחת',
 'לנהיוארליייו',
 'והאריאליאהוכ',
 'טבידאהםיהווי',
 'השוקאלילויםל',
 'שקאיהיירוהבר',
 'ואדימאהםהריב',
 'והיוהמדלימםמ']

In [166]:
type(X_samples)

list

In [157]:
X_sequences = tokenizer.texts_to_sequences(X_samples)

In [159]:
X_sequences[:10]

[[8, 4, 1, 8, 4, 5, 1, 4, 3, 10, 12, 4],
 [3, 7, 2, 4, 26, 1, 3, 3, 2, 3, 2, 9],
 [11, 19, 1, 3, 12, 7, 15, 5, 1, 10, 15, 6],
 [5, 13, 3, 1, 2, 4, 7, 5, 1, 1, 1, 2],
 [2, 3, 4, 7, 1, 4, 5, 1, 4, 3, 2, 14],
 [25, 8, 1, 16, 4, 3, 12, 1, 3, 2, 2, 1],
 [3, 9, 2, 17, 4, 5, 1, 5, 2, 1, 12, 5],
 [9, 17, 4, 1, 3, 1, 1, 7, 2, 3, 8, 7],
 [2, 4, 16, 1, 10, 4, 3, 12, 3, 7, 1, 8],
 [2, 3, 1, 2, 3, 10, 16, 5, 1, 10, 12, 10]]

In [160]:
X_embedding = []

for s in X_sequences:
  if len(s) >= 12:
    X_embedding.append(s[:12])
  else:
    X_embedding.append(s + [0]*(max_len-len(s)))

X_embedding = np.array(X_embedding)

In [161]:
X_embedding[:10]

array([[ 8,  4,  1,  8,  4,  5,  1,  4,  3, 10, 12,  4],
       [ 3,  7,  2,  4, 26,  1,  3,  3,  2,  3,  2,  9],
       [11, 19,  1,  3, 12,  7, 15,  5,  1, 10, 15,  6],
       [ 5, 13,  3,  1,  2,  4,  7,  5,  1,  1,  1,  2],
       [ 2,  3,  4,  7,  1,  4,  5,  1,  4,  3,  2, 14],
       [25,  8,  1, 16,  4,  3, 12,  1,  3,  2,  2,  1],
       [ 3,  9,  2, 17,  4,  5,  1,  5,  2,  1, 12,  5],
       [ 9, 17,  4,  1,  3,  1,  1,  7,  2,  3,  8,  7],
       [ 2,  4, 16,  1, 10,  4,  3, 12,  3,  7,  1,  8],
       [ 2,  3,  1,  2,  3, 10, 16,  5,  1, 10, 12, 10]])

In [163]:
X_predictions = model.predict(X_embedding)

396/396 [==============================] - 1s 3ms/step


In [164]:
type(X_predictions)

numpy.ndarray

In [165]:
X_predictions[:10]

array([[0.89266473],
       [0.29751754],
       [0.04814423],
       [0.14317091],
       [0.73630244],
       [0.7349972 ],
       [0.483839  ],
       [0.83907604],
       [0.95200855],
       [0.6789549 ]], dtype=float32)

In [181]:
test2_output = pd.concat([pd.Series(X_samples), pd.Series(map(lambda x: x[0], X_predictions)), phrases_12_df['start_index'], phrases_12_df['end_index']], axis=1)

In [182]:
test2_output['prediction'] = test2_output[1].round(0).astype('int')

In [183]:
test2_output.columns = ['text', 'pred_prob','start_index', 'end_index', 'prediction']

In [184]:
test2_output = test2_output[['text', 'prediction', 'pred_prob','start_index', 'end_index']]

In [185]:
test2_output.head()

,text,prediction,pred_prob,start_index,end_index
0,באיבאליאהמםא,1,0.892665,0,12
1,הרואץיההוהוש,0,0.297518,24,36
2,עפיהםרחלימחת,0,0.048144,48,60
3,לנהיוארליייו,0,0.143171,72,84
4,והאריאליאהוכ,1,0.736302,96,108


In [186]:
from google.colab import files

test2_output.to_csv('RNNtest2.csv', index=False ,encoding = 'utf-8-sig') 
files.download('RNNtest2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
[l for l in 'ויאמריהוהאלמשה']

['ו', 'י', 'א', 'מ', 'ר', 'י', 'ה', 'ו', 'ה', 'א', 'ל', 'מ', 'ש', 'ה']

In [59]:
[word_index[l] for l in 'ויאמריהוהאלמשה']

[2, 1, 4, 10, 7, 1, 3, 2, 3, 4, 5, 10, 9, 3]

In [78]:
string = 'וילךמשהלישראל'

In [79]:
predict_me = [[word_index[l] for l in string]]

In [80]:
model.predict(predict_me)

1/1 [==============================] - 0s 26ms/step


array([[0.98409086]], dtype=float32)